In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
from tabulate import tabulate
import pyarrow as pa
import pyarrow.parquet as pq
import json
import pickle as pkl
import plotly.graph_objects as go
from plotting import CandlePlot
pd.set_option("display.max_columns", None)

In [ ]:
class Data:
    
    def __init__(self, source):
        assert type(source) == str or type(source) == pd.DataFrame, 'Invalid source'
        if type(source) == str:
            self.df = {
                'raw': pd.read_pickle(source)
            }
        elif type(source) == pd.DataFrame:
            self.df = {
                'raw': source.copy()
            }            

        if 'time' in self.df['raw'].columns:
            self.df['raw']['time'] = [ x.replace(tzinfo=None) for x in self.df['raw']['time']]
        self.datalen = self.df['raw'].shape[0]

    def __repr__(self) -> str:
        repr = str()
        for name, df in self.df.items():
            repr = repr + name + ':\n' + str(pd.concat([df.head(2), df.tail(1)])) + '\n'
        return repr

    def prep_data(self, name: str, start: int, end: int, source: str='raw', cols: list=None):
        '''Create new dataframe with specified list of columns and number of rows as preparation for fast data creation
        '''
        # assert (direction != 1 or direction != -1), 'direction must be 1 (top) or -1 (bottom)'

        if cols == None:
            cols = self.df[source].columns
        # if direction == 1:
        #     self.df[name] = self.df[source][cols].iloc[:rows].copy()
        # else:
        #     self.df[name] = self.df[source][cols].iloc[-rows:].copy()
        if start == None:
            start = 0
        if end == None:
            end = self.datalen

        assert end > start, f'start={start}, end={end} not valid'

        self.df[name] = self.df[source][cols].iloc[start:end].copy()
        self.df[name].reset_index(drop=True, inplace=True)

    def add_columns(self, name: str, cols: dict):
        '''Add new columns to component dataframes
        '''        
        exist_cols = list(self.df[name].columns)
        # cols = exist_cols + cols
        for col, _type in cols.items():
            self.df[name][col] = pd.Series(dtype=_type) #self.df[name][col].apply(_type)
        # self.df[name] = self.df[name].reindex(columns = cols) 

    def prepare_fast_data(self, name: str, start: int, end: int, source: str='raw', cols: list=None, add_cols: dict=None):
        '''Prepare data as an array for fast processing
        fcols = {col1: col1_index, col2: col2_index, .... }     
        fastdf = [array[col1], array[col2], array[col3], .... ]
        Accessed by: self.fdata()
        '''

        self.prep_data(name=name, start=start, end=end, source=source, cols=cols)
        if add_cols:
            # types = dict() if types is None else types
            self.add_columns(name=name, cols=add_cols)

        self.fcols = dict()
        for i in range(len(self.df[name].columns)):
            self.fcols[self.df[name].columns[i]] = i
        self.fastdf = [self.df[name][col].array for col in self.df[name].columns]
        self.fdatalen = len(self.fastdf[0])

    def fdata(self, column: str=None, index: int=None, rows: int=None):
        if column is None:
            return self.fastdf
        if index is None:
            return self.fastdf[self.fcols[column]]
        else:
            if rows:
                try:
                    return self.fastdf[self.fcols[column]][index:index+rows]
                except:
                    return self.fastdf[self.fcols[column]][index:]
            else:
                return self.fastdf[self.fcols[column]][index]
        
    def update_fdata(self, column: str, index: int=None, value: float=None):
        assert value is not None, 'Value cannot be null'
        if index is None:
            assert len(value) == self.fdatalen
            for i in range(self.fdatalen):
                self.fastdf[self.fcols[column]][i] = value[i]
                print(i, )
        else:
            self.fastdf[self.fcols[column]][index] = value

In [ ]:
with open("../data/instruments.json", 'r') as f:
    instr = json.load(f)

In [ ]:
ticker = 'EUR_GBP'

In [ ]:
instr[ticker]

In [ ]:
# df = pd.read_pickle(f"../data/{ticker}_M5.pkl")

In [ ]:
# df['pip_return'] = (df.mid_c - df.mid_c.shift(1)) * pow(10, -instr[ticker]['pipLocation'])

In [ ]:
# pd.concat([df.head(3), df.tail(3)])

In [ ]:
# our_cols = ['time', 'mid_o', 'mid_c', 'bid_o', 'ask_o']

In [ ]:
# d = Data(df[our_cols])
# d

In [ ]:
SIZE, ENTRY, TP, SL = 0, 1, 2, 3
EXIT, PIPS = 2, 3

In [ ]:
def read_data(ticker: str, frequency: str):
    our_cols = ['time', 'mid_c', 'bid_c', 'ask_c']
    df = pd.read_pickle(f"../data/{ticker}_{frequency}.pkl")
    return Data(df[our_cols])

In [ ]:
def open_trades(d: Data, i: int, tp_pips: int, sl_pips: int, trade_size: int, trade_no: int):
    long_tp = d.fdata('mid_c', i) + tp_pips * pow(10, instr[ticker]['pipLocation'])
    short_tp = d.fdata('mid_c', i) - tp_pips * pow(10, instr[ticker]['pipLocation'])

    long_sl = d.fdata('mid_c', i) - sl_pips * pow(10, instr[ticker]['pipLocation'])
    short_sl = d.fdata('mid_c', i) + sl_pips * pow(10, instr[ticker]['pipLocation'])

    if i == 0:
        open_longs, open_shorts = dict(), dict()
    else:
        open_longs, open_shorts = d.fdata('open_longs', i-1).copy(), d.fdata('open_shorts', i-1).copy()
        # open_longs, open_shorts = json.loads(d.fdata('open_longs', i-1)), json.loads(d.fdata('open_shorts', i-1))

    open_longs[trade_no] = (trade_size, d.fdata('bid_c', i), long_tp, long_sl) # (SIZE, ENTRY, TP, SL)
    d.update_fdata('open_longs', i, open_longs)
    # d.update_fdata('open_longs', i, str(open_longs).replace("'",'"'))
    open_shorts[trade_no] = (trade_size, d.fdata('ask_c', i), short_tp, short_sl) # (SIZE, ENTRY, TP, SL)
    d.update_fdata('open_shorts', i, open_shorts)
    # d.update_fdata('open_shorts', i, str(open_shorts).replace("'",'"'))

    d.update_fdata('new_trade_no', i, trade_no)
    
    return long_tp, short_tp # equals next_up_grid, next_down_grid for next grid level trades

In [ ]:
def close_long(d: Data, i: int, trade_no: int):
    # Remove from open longs
    open_longs = d.fdata('open_longs', i).copy()
    closing_long = open_longs[trade_no]
    del open_longs[trade_no]
    d.update_fdata('open_longs', i, open_longs)

    # Append to closed longs
    pips = (d.fdata('ask_c', i) - closing_long[ENTRY]) * pow(10, instr[ticker]['pipLocation'])
    # if type(d.fdata('closed_longs', i)) != dict:
    #     closed_longs = dict()
    # else:
    #     closed_longs = d.fdata('closed_longs', i).copy()
    closed_longs = d.fdata('closed_longs', i).copy() if type(d.fdata('closed_longs', i)) == dict else dict()
    closed_longs[trade_no] = (closing_long[SIZE], closing_long[ENTRY], d.fdata('ask_c', i), pips) # (SIZE, ENTRY, EXIT, PIPS)

    d.update_fdata('closed_longs', i, closed_longs)

In [ ]:
def close_short(d: Data, i: int, trade_no: int):
    # Remove from open shorts
    open_shorts = d.fdata('open_shorts', i).copy()
    closing_short = open_shorts[trade_no]
    del open_shorts[trade_no]
    d.update_fdata('open_shorts', i, open_shorts)

    # Append to closed shorts
    pips = (closing_short[ENTRY] - d.fdata('bid_c', i)) * pow(10, instr[ticker]['pipLocation'])
    # if type(d.fdata('closed_shorts', i)) != dict:
    #     closed_shorts = dict()
    # else:
    #     closed_shorts = d.fdata('closed_shorts', i).copy()
    closed_shorts = d.fdata('closed_shorts', i).copy() if type(d.fdata('closed_shorts', i)) == dict else dict()
    closed_shorts[trade_no] = (closing_short[SIZE], closing_short[ENTRY], d.fdata('bid_c', i), pips) # (SIZE, ENTRY, EXIT, PIPS)

    d.update_fdata('closed_shorts', i, closed_shorts)

In [ ]:
def cum_long_position(d: Data, i: int):
    open_longs = d.fdata('open_longs', i).copy() if type(d.fdata('open_longs', i)) == dict else dict()
    cum_long_position = 0
    for _, trade in open_longs.items():
        cum_long_position = cum_long_position + trade[SIZE]
    return cum_long_position

def cum_short_position(d: Data, i: int):
    open_shorts = d.fdata('open_shorts', i).copy() if type(d.fdata('open_shorts', i)) == dict else dict()
    cum_short_position = 0
    for _, trade in open_shorts.items():
        cum_short_position = cum_short_position + trade[SIZE]
    return cum_short_position

def unrealised_pnl(d: Data, i: int):
    open_longs = d.fdata('open_longs', i).copy() if type(d.fdata('open_longs', i)) == dict else dict()
    open_shorts = d.fdata('open_shorts', i).copy() if type(d.fdata('open_shorts', i)) == dict else dict()
    pnl = 0
    for _, trade in open_longs.items():
        pnl = pnl + trade[SIZE] * (d.fdata('mid_c', i) - trade[ENTRY])
    for _, trade in open_shorts.items():
        pnl = pnl + trade[SIZE] * (trade[ENTRY] - d.fdata('mid_c', i))
    return pnl

def realised_pnl(d: Data, i: int):
    closed_longs = d.fdata('closed_longs', i).copy() if type(d.fdata('closed_longs', i)) == dict else dict()
    closed_shorts = d.fdata('closed_shorts', i).copy() if type(d.fdata('closed_shorts', i)) == dict else dict()
    pnl = 0
    for _, trade in closed_longs.items():
        pnl = pnl + trade[SIZE] * (trade[EXIT] - trade[ENTRY])
    for _, trade in closed_shorts.items():
        pnl = pnl + trade[SIZE] * (trade[ENTRY] - trade[EXIT])
    return pnl

In [ ]:
def current_values(d, i):
    _cum_long_position = cum_long_position(d, i)
    _cum_short_position = cum_short_position(d, i)
    _unrealised_pnl = unrealised_pnl(d, i)
    _realised_pnl = realised_pnl(d, i)
    ac_bal = d.fdata('ac_bal', i-1) + _realised_pnl
    margin_used = (_cum_long_position + _cum_short_position) * float(instr[ticker]['marginRate'])
    margin_closeout = ac_bal + _unrealised_pnl
    return margin_used, margin_closeout

def stop_loss_oldest(d: Data, i: int):
    margin_used, margin_closeout = current_values(d, i)
    # Margin call is triggered when margin closeout is less than 50% of margin used.
    # Here stop loss is triggered if it falls below 60%
    if margin_closeout < margin_used * 0.6:
        reduced_margin = margin_closeout / 0.6
        while reduced_margin < margin_used:
            longs = list(d.fdata('open_longs', i).keys())
            shorts = list(d.fdata('open_shorts', i).keys())
            oldest_long = longs[0] if len(longs) > 0 else None
            oldest_short = shorts[0] if len(shorts) > 0 else None
            if oldest_long == None and oldest_short == None:
                break
            elif oldest_long == None:
                close_short(d, i, oldest_short)
                margin_used, _ = current_values(d, i)
                print(f'stop loss: {i}, short: {oldest_short}')
            elif oldest_short == None:
                close_long(d, i, oldest_long)
                margin_used, _ = current_values(d, i)
                print(f'stop loss: {i}, long: {oldest_long}')
            else:
                if oldest_long <= oldest_short:
                    close_long(d, i, oldest_long)
                    margin_used, _ = current_values(d, i)
                    print(f'stop loss: {i}, long: {oldest_long}')
                else:
                    close_short(d, i, oldest_short)
                    margin_used, _ = current_values(d, i)
                    print(f'stop loss: {i}, short: {oldest_short}')

def stop_loss_farthest(d: Data, i: int):
    margin_used, margin_closeout = current_values(d, i)
    # Margin call is triggered when margin closeout is less than 50% of margin used.
    # Here stop loss is triggered if it falls below 60%
    price = d.fdata('mid_c', i)
    if margin_closeout < margin_used * 0.6:
        reduced_margin = margin_closeout / 0.6
        while reduced_margin < margin_used:
            farthest_long_price, farthest_short_price = price, price
            farthest_long, farthest_short = None, None
            for long, trade in d.fdata('open_longs', i).items():
                if trade[ENTRY] > farthest_long_price:
                    farthest_long_price = trade[ENTRY]
                    farthest_long = long
            for short, trade in d.fdata('open_shorts', i).items():
                if trade[ENTRY] < farthest_short_price:
                    farthest_short_price = trade[ENTRY]
                    farthest_short = short
            if farthest_long == None and farthest_short == None:
                break
            elif farthest_long == None:
                close_short(d, i, farthest_short)
                margin_used, _ = current_values(d, i)
                print(f'stop loss: {i}, short: {farthest_short}')
            elif farthest_short == None:
                close_long(d, i, farthest_long)
                margin_used, _ = current_values(d, i)
                print(f'stop loss: {i}, long: {farthest_long}')
            else:
                if farthest_long_price - price > price - farthest_short_price:
                    close_long(d, i, farthest_long)
                    margin_used, _ = current_values(d, i)
                    print(f'stop loss: {i}, long: {farthest_long}')
                else:
                    close_short(d, i, farthest_short)
                    margin_used, _ = current_values(d, i)
                    print(f'stop loss: {i}, short: {farthest_short}')

In [ ]:
def calc_ac_values(d: Data, i: int, init_bal: float=None):
    d.update_fdata('cum_long_position', i, cum_long_position(d, i))
    d.update_fdata('cum_short_position', i, cum_short_position(d, i))
    d.update_fdata('unrealised_pnl', i, unrealised_pnl(d, i))
    d.update_fdata('realised_pnl', i, realised_pnl(d, i))
    # First candle
    if i == 0:               
        d.update_fdata('ac_bal', i, init_bal)
    # Subsequent candles
    else:
        d.update_fdata('ac_bal', i, d.fdata('ac_bal', i-1) + d.fdata('realised_pnl', i))
    d.update_fdata('margin_used', i, \
                (d.fdata('cum_long_position', i) + d.fdata('cum_short_position', i)) * float(instr[ticker]['marginRate']))
    d.update_fdata('margin_closeout', i, d.fdata('ac_bal', i) + d.fdata('unrealised_pnl', i))

In [ ]:
def run_sim(d: Data, sim_name: str, start: int, end: int, init_bal: int, init_trade_size: int, grid_pips: int, sl_grid_count: int) -> pd.DataFrame:
    tp_pips = grid_pips
    sl_pips = grid_pips * sl_grid_count
    
    add_cols = dict(
        open_longs=object,
        open_shorts=object,
        new_trade_no=int,
        closed_longs=object,
        closed_shorts=object,
        cum_long_position=int,
        cum_short_position=int,
        unrealised_pnl=float,
        realised_pnl=float,
        ac_bal=float,
        margin_used=float,
        margin_closeout=float
    )
    d.prepare_fast_data(name=sim_name, start=start, end=end, add_cols=add_cols)

    for i in tqdm(range(d.fdatalen), desc=" Simulating... "):       
        # First candle
        if i == 0:
            # Open trades
            trade_no = 1            
            next_up_grid, next_down_grid = open_trades(d=d, i=i, tp_pips=tp_pips, sl_pips=sl_pips, trade_size=init_trade_size, trade_no=trade_no)
            calc_ac_values(d, i, init_bal)
        # Subsequent candles
        else:
            # Open trades
            if d.fdata('mid_c', i) >= next_up_grid or d.fdata('mid_c', i) <= next_down_grid:
                trade_no = trade_no + 1            
                next_up_grid, next_down_grid = open_trades(d=d, i=i, tp_pips=tp_pips, sl_pips=sl_pips, trade_size=init_trade_size, trade_no=trade_no)
            else: # Cascade open positions from prev candle to current candle
                d.update_fdata('open_longs', i, d.fdata('open_longs', i-1).copy())
                d.update_fdata('open_shorts', i, d.fdata('open_shorts', i-1).copy())

            # Take profit long positions
            open_longs = d.fdata('open_longs', i).copy()
            for trade_no, trade in open_longs.items():
                if d.fdata('mid_c', i) >= trade[TP]: # or d.fdata('mid_c', i) <= trade[SL]:
                    close_long(d, i, trade_no)

            # Take profit short positions
            open_shorts = d.fdata('open_shorts', i).copy()
            for trade_no, trade in open_shorts.items():
                if d.fdata('mid_c', i) <= trade[TP]: # or d.fdata('mid_c', i) >= trade[SL]:
                    close_short(d, i, trade_no)

            stop_loss_farthest(d, i)
            calc_ac_values(d, i)

    return dict(
        sim_name = sim_name,
        init_trade_size=init_trade_size,
        grid_pips=grid_pips,
        sl_grid_count=sl_grid_count,
        results = d.df[sim_name]
    )

In [ ]:
def process_sim(d: Data, ticker: str, frequency: str, counter: int, start: int, end:int, init_bal: int, init_trade_size: int, grid_pips: int, sl_grid_count: int, inputs_list: list, inputs_file: str):
    sim_name = f'{ticker}-{frequency}-{counter}'
    header = ['sim_name', 'init_trade_size', 'grid_pips', 'sl_grid_count', 'stoploss_pips']
    inputs = [sim_name, init_trade_size, grid_pips, sl_grid_count, grid_pips * sl_grid_count]
    print(tabulate([inputs], header, tablefmt='plain'))
    result= run_sim(
        d=d,
        sim_name=sim_name,
        start=start,
        end=end,
        init_bal=init_bal,
        init_trade_size=init_trade_size,
        grid_pips=grid_pips,
        sl_grid_count=sl_grid_count
    )
    # print('Saving files...')
    with open(f'D:/Trading/ml4t-data/grid/{sim_name}.pkl', 'wb') as file:
        pkl.dump(result, file)

    inputs_list.append(inputs)
    inputs_df = pd.DataFrame(inputs_list, columns=header)
    inputs_df.to_pickle(f'D:/Trading/ml4t-data/grid/{ticker}-{frequency}-' + inputs_file)
    
    del d.df[sim_name]
    counter =  counter + 1
    return inputs

In [ ]:
# periods = [None] # all candles
# init_bal = [500, 1000, 2000, 5000]
# trade_size = [10, 100, 1000, 10000]
# grid_pips = [20, 30, 40, 50, 60, 100]
# stoploss_lvl = [5, 10, 15, 20, 30, 50, 100, 10000]
# INPUTS_FILE = 'inputs.pkl'

In [ ]:
checkpoint = 1
counter = 1
start = None
end = None
tickers = ['EUR_USD']
frequency = ['M5']
init_bal = [1000]
init_trade_size = [1000]
grid_pips = [20]
sl_grid_count = [40]
INPUTS_FILE = 'inputs.pkl'

In [ ]:
inputs_list = list()
for tk in tickers:
    for f in frequency:
        for ib in init_bal:
            for t in init_trade_size:
                for g in grid_pips:
                    for sl in sl_grid_count:
                        if counter >= checkpoint:
                            d = read_data(ticker, f)
                            inputs = process_sim(
                                d=d,
                                ticker=tk,
                                frequency=f,
                                counter=counter,
                                start=start,
                                end=end,
                                init_bal=ib,
                                init_trade_size=t,
                                grid_pips=g,
                                sl_grid_count=sl,
                                inputs_list=inputs_list,
                                inputs_file=INPUTS_FILE
                            )  
                        counter =  counter + 1

In [ ]:
inputs

In [ ]:
results = pd.read_pickle(f'D:/Trading/ml4t-data/grid/EUR_USD-M5-1.pkl')['results'].copy()

In [ ]:
results['net_bal'] = results['ac_bal'] + results['unrealised_pnl']
results['cum_position'] = results['cum_long_position'] + results['cum_short_position']


In [ ]:
results

In [ ]:
results[~results['realised_pnl'].isnull()]

In [ ]:
results.info()

In [ ]:
cp = CandlePlot(results, candles=False)
cp.show_plot(line_traces=['mid_c'])

In [ ]:
cp = CandlePlot(results, candles=False)
cp.show_plot(line_traces=['ac_bal', 'net_bal', 'unrealised_pnl'])

In [ ]:
cp = CandlePlot(results, candles=False)
cp.show_plot(line_traces=['margin_used'])

In [ ]:
cp = CandlePlot(results, candles=False)
cp.show_plot(line_traces=['cum_long_position', 'cum_short_position', 'cum_position'])

In [ ]:
results['margin_used_50'] = results['margin_used'] * 0.5
results['margin_call'] = results['margin_used_50'] > results['margin_closeout']

In [ ]:
cp = CandlePlot(results, candles=False)
cp.show_plot(line_traces=['margin_call'])

In [ ]:
cp = CandlePlot(results, candles=False)
cp.show_plot(line_traces=['margin_used_50', 'margin_closeout'])